In [2]:
import json
import os
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr
import h5py

### Setup

In [3]:
os.chdir('..')
os.getcwd()

'c:\\Users\\delgr\\Projects\\advancedPython\\Day6a'

### Set Script Parameters

In [4]:
session_dir = './data/extracted/20161214_cori_steinmetz'

In [5]:
session_path = Path(session_dir)
assert session_path.exists()

### Load Data

#### Trials

In [6]:
trials = pd.DataFrame(np.load(session_path / 'trials/trials.npy'),)
trials.set_index('trial', inplace=True)
trials

,contrast_left,contrast_right,stim_onset,licks,response,feedback,gocue,response_time
trial,,,,,,,,
1,100,0,500,11,b'right',b'pos',1027.0,1610.0
2,0,50,3000,12,b'left',b'pos',3374.0,3070.0
3,100,50,5500,13,b'right',b'pos',5825.0,6170.0
4,0,0,8000,0,b'neutral',b'pos',8261.0,10130.0
5,50,100,10500,7,b'right',b'neg',10662.0,NaN
...,...,...,...,...,...,...,...,...
360,50,25,898000,0,b'None',b'None',NaN,NaN
361,50,25,900500,0,b'None',b'None',NaN,NaN
362,0,50,903000,0,b'None',b'None',NaN,NaN


In [7]:
trials_da = xr.Dataset.from_dataframe(trials)
trials_da

<xarray.Dataset> Size: 16kB
Dimensions:         (trial: 364)
Coordinates:
  * trial           (trial) uint16 728B 1 2 3 4 5 6 ... 359 360 361 362 363 364
Data variables:
    contrast_left   (trial) uint8 364B 100 0 100 0 50 0 0 ... 100 50 50 0 25 100
    contrast_right  (trial) uint8 364B 0 50 50 0 100 0 0 ... 100 25 25 50 0 100
    stim_onset      (trial) int64 3kB 500 3000 5500 ... 903000 905500 908000
    licks           (trial) uint8 364B 11 12 13 0 7 3 0 0 0 ... 0 0 0 0 0 0 0 0
    response        (trial) object 3kB b'right' b'left' ... b'None' b'None'
    feedback        (trial) object 3kB b'pos' b'pos' b'pos' ... b'None' b'None'
    gocue           (trial) float64 3kB 1.027e+03 3.374e+03 ... nan nan
    response_time   (trial) float64 3kB 1.61e+03 3.07e+03 6.17e+03 ... nan nan

#### Wheel Movement

In [53]:
wheel = np.load(session_path / 'wheel/wheel.npy')
time = np.load(session_path / 'wheel/time.npy')
wheel_trials = wheel.reshape(len(trials), -1)

In [58]:

wheel_da = xr.DataArray(wheel_trials, coords={'trial': np.arange(len(trials)), 'time': time[:wheel_trials.shape[-1]]}, name='wheel')
wheel_da.attrs = json.loads((session_path / 'wheel/wheel.json').read_text())
# wheel_da.time.attrs = json.loads((session_path / 'wheel/time.json').read_text())
wheel_da

<xarray.DataArray 'wheel' (trial: 364, time: 250)> Size: 91kB
array([[-1,  0,  0, ...,  1,  0,  1],
       [ 0, -1,  0, ...,  1,  0,  0],
       [ 0,  0, -1, ..., -1,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], shape=(364, 250), dtype=int8)
Coordinates:
  * trial    (trial) int64 3kB 0 1 2 3 4 5 6 7 ... 357 358 359 360 361 362 363
  * time     (time) uint16 500B 0 10 20 30 40 50 ... 2450 2460 2470 2480 2490
Attributes:
    notes:    Positive is rightward movement.

#### LFP Recording

In [9]:
lfp = np.load(session_path / 'lfp/recording.npy')
lfp_trials = lfp.reshape(len(trials), -1, lfp.shape[-1])
lfp_trials.shape

(364, 250, 7)

In [10]:
channels = pd.read_csv(session_path / 'lfp/channels.csv')
channels

,Channel,BrainRegion
0,CHAN001,ACA
1,CHAN002,LS
2,CHAN003,MOs
3,CHAN004,CA3
4,CHAN005,DG
5,CHAN006,SUB
6,CHAN007,VISp


In [11]:
lfp_times = np.load(session_path / 'lfp/times.npy')
lfp_times

array([     0,     10,     20, ..., 909970, 909980, 909990],
      shape=(91000,), dtype=uint32)

In [12]:
lfp_trials.shape

(364, 250, 7)

In [13]:
lfp_da = xr.DataArray(
    lfp_trials,
    coords=[np.arange(len(trials)), lfp_times[:lfp_trials.shape[1]], channels['Channel'].values.astype('S')],
    dims=('trial', 'time', 'channel'),
    name='lfp'
)
lfp_da

<xarray.DataArray 'lfp' (trial: 364, time: 250, channel: 7)> Size: 5MB
array([[[-2.85079365e+00,  9.41496599e-01,  4.37979592e+00, ...,
          2.18208353e+01,  6.71088435e+00,  1.12764378e+00],
        [-4.03968254e+00, -4.18367347e-02, -3.55020408e+00, ...,
         -5.10939725e+00,  5.51088435e+00, -5.85417440e+00],
        [-4.19523810e+00, -8.50850340e+00, -1.47020408e+00, ...,
         -4.65590888e+00, -5.38911565e+00, -7.71781076e+00],
        ...,
        [ 7.26984127e-01,  4.52482993e+00,  1.97979592e+00, ...,
          2.22975795e+01,  8.67755102e+00,  1.09458256e+01],
        [-2.21746032e+00, -1.68367347e-02, -1.41020408e+00, ...,
         -3.76839108e-01,  1.15942177e+01,  5.78218924e+00],
        [ 1.09936508e+01,  8.23316327e+00,  8.79979592e+00, ...,
          1.04952539e+01,  2.41942177e+01,  1.44640074e+01]],

       [[ 9.14263039e+00,  8.85068027e+00,  8.74408163e+00, ...,
         -1.95609872e+00, -8.68435374e+00, -3.92912801e+00],
        [ 1.14759637e+01,  7.84234694e+00,  1.13540816e+01, ...,
          1.32297105e+00, -1.48435374e+00, -6.65640074e+00],
        [ 1.47648526e+01,  7.06734694e+00,  1.41040816e+01, ...,
          7.03227337e+00, -1.05102041e+00,  1.47996289e+00],
...
         -1.42705268e+01, -1.76870748e-02,  9.79220779e-01],
        [ 1.96126984e+01,  8.68367347e+00,  1.27561224e+01, ...,
          2.99240626e-01,  1.30823129e+01,  1.14064935e+01],
        [ 7.19047619e+00,  3.62534014e+00, -3.13877551e-01, ...,
         -1.66587565e+00,  7.11564626e+00,  4.43376623e+00]],

       [[ 3.18326531e+01,  1.86253401e+01,  9.44387755e+00, ...,
          2.38611770e+01,  1.50935374e+01,  5.38868275e+00],
        [ 2.46215420e+01,  1.20336735e+01,  3.15387755e+00, ...,
         -2.24347413e+00,  6.76020408e+00, -3.72040816e+00],
        [ 1.04659864e+01,  5.75034014e+00, -2.59612245e+00, ...,
         -1.44760323e+01,  2.27687075e+00, -4.64768089e+00],
        ...,
        [-5.93401361e+00, -6.66326531e-01, -3.64612245e+00, ...,
          2.57216421e+01,  3.46020408e+00,  1.55523191e+01],
        [-1.26451247e+01, -7.89965986e+00, -4.52612245e+00, ...,
          1.13262933e+01,  1.41102041e+01,  1.60432282e+01],
        [-1.59229025e+01, -1.74746599e+01, -7.18612245e+00, ...,
          1.80239677e+01, -5.80646259e+00,  5.57050093e+00]]],
      shape=(364, 250, 7))
Coordinates:
  * trial    (trial) int64 3kB 0 1 2 3 4 5 6 7 ... 357 358 359 360 361 362 363
  * time     (time) uint32 1kB 0 10 20 30 40 50 ... 2450 2460 2470 2480 2490
  * channel  (channel) |S7 49B b'CHAN001' b'CHAN002' ... b'CHAN006' b'CHAN007'

In [14]:
brain_region_da = xr.DataArray(
    channels['BrainRegion'].values,
    coords=[channels['Channel'].values.astype('S')],
    dims=('channel',),
    name='brain_region',
)
brain_region_da

<xarray.DataArray 'brain_region' (channel: 7)> Size: 56B
array(['ACA', 'LS', 'MOs', 'CA3', 'DG', 'SUB', 'VISp'], dtype=object)
Coordinates:
  * channel  (channel) |S7 49B b'CHAN001' b'CHAN002' ... b'CHAN006' b'CHAN007'

#### Eye Tracking

In [17]:
eye = np.load(session_path / 'eye_tracking/recording.npy')
eye

array([(     0,  0.81285601, 0.66424745, 0.16584056),
       (    10,  0.77816193, 0.6127516 , 0.15866767),
       (    20,  0.81235208, 0.63276851, 0.16966705), ...,
       (909970, -0.45360652, 0.13045109, 0.11383507),
       (909980, -0.48602562, 0.23918371, 0.10993529),
       (909990, -0.49164412, 0.27000163, 0.11992919)],
      shape=(91000,), dtype=[('time', '<u4'), ('pupil_x', '<f8'), ('pupil_y', '<f8'), ('pupil_area', '<f8')])

In [36]:
eye_mdata = json.loads((session_path / 'eye_tracking/recording.json').read_text())
eye_mdata

{'pupil_x': {'units': 'px', 'description': 'Pupil X Position'},
 'pupil_y': {'units': 'px', 'description': 'Pupil Y Position'},
 'pupil_area': {'units': 'px^2', 'description': 'Pupil Area'}}

In [37]:


pupil_x = xr.DataArray(
    eye['pupil_x'].reshape(len(trials), -1),
    coords=[np.arange(len(trials)), lfp_times[:lfp_trials.shape[1]]],
    dims=('trial', 'time')
)
pupil_x.attrs = eye_mdata['pupil_x']
pupil_x


<xarray.DataArray (trial: 364, time: 250)> Size: 728kB
array([[ 0.81285601,  0.77816193,  0.81235208, ...,  0.7957129 ,
         0.85481015,  0.9076376 ],
       [ 0.94817386,  0.95962932,  0.95623047, ...,  0.38599669,
         0.34658371,  0.30899389],
       [ 0.3880897 ,  0.3856168 ,  0.32983245, ...,  0.52308137,
         0.67938778,  0.79211687],
       ...,
       [ 0.21872131,  0.14844803,  0.14106962, ...,  0.03984695,
         0.1192567 ,  0.03193037],
       [-0.23057746, -0.1832362 , -0.3408799 , ..., -0.30211522,
        -0.26709205, -0.30083151],
       [-0.39643424, -0.45370558, -0.45053439, ..., -0.45360652,
        -0.48602562, -0.49164412]], shape=(364, 250))
Coordinates:
  * trial    (trial) int64 3kB 0 1 2 3 4 5 6 7 ... 357 358 359 360 361 362 363
  * time     (time) uint32 1kB 0 10 20 30 40 50 ... 2450 2460 2470 2480 2490
Attributes:
    units:        px
    description:  Pupil X Position

In [39]:

pupil_y = xr.DataArray(
    eye['pupil_y'].reshape(len(trials), -1),
    coords=[np.arange(len(trials)), lfp_times[:lfp_trials.shape[1]]],
    dims=('trial', 'time')
)
pupil_y.attrs = eye_mdata['pupil_y']

pupil_y


<xarray.DataArray (trial: 364, time: 250)> Size: 728kB
array([[ 0.66424745,  0.6127516 ,  0.63276851, ...,  0.37188813,
         0.23835723,  0.23160477],
       [ 1.23273747,  1.1461164 ,  1.07586317, ..., -0.90672734,
        -0.76225968, -0.7643367 ],
       [-0.54593809, -0.54971865, -0.53757296, ..., -0.25438543,
        -0.10055672,  0.02068106],
       ...,
       [ 1.07398729,  1.21567852,  1.14906106, ...,  1.07504472,
         1.12505063,  1.14535118],
       [ 0.23600561,  0.19862846,  0.19627369, ...,  0.12601246,
         0.06920929,  0.06199038],
       [-0.11972398, -0.14335603, -0.12658565, ...,  0.13045109,
         0.23918371,  0.27000163]], shape=(364, 250))
Coordinates:
  * trial    (trial) int64 3kB 0 1 2 3 4 5 6 7 ... 357 358 359 360 361 362 363
  * time     (time) uint32 1kB 0 10 20 30 40 50 ... 2450 2460 2470 2480 2490
Attributes:
    units:        px
    description:  Pupil Y Position

In [40]:
pupil_area = xr.DataArray(
    eye['pupil_area'].reshape(len(trials), -1),
    coords=[np.arange(len(trials)), lfp_times[:lfp_trials.shape[1]]],
    dims=('trial', 'time')
)
pupil_area.attrs = eye_mdata['pupil_area']
pupil_area

<xarray.DataArray (trial: 364, time: 250)> Size: 728kB
array([[0.16584056, 0.15866767, 0.16966705, ..., 0.16423223, 0.17261705,
        0.17060842],
       [0.17180513, 0.16491603, 0.15992441, ..., 0.21022576, 0.20026623,
        0.19983341],
       [0.15330889, 0.14533824, 0.14872697, ..., 0.12516653, 0.14728813,
        0.13150498],
       ...,
       [0.04059704, 0.05748408, 0.05283589, ..., 0.09169307, 0.08196067,
        0.09649467],
       [0.0453804 , 0.04017045, 0.06644672, ..., 0.1050599 , 0.10896764,
        0.09869005],
       [0.07131883, 0.0601189 , 0.06215936, ..., 0.11383507, 0.10993529,
        0.11992919]], shape=(364, 250))
Coordinates:
  * trial    (trial) int64 3kB 0 1 2 3 4 5 6 7 ... 357 358 359 360 361 362 363
  * time     (time) uint32 1kB 0 10 20 30 40 50 ... 2450 2460 2470 2480 2490
Attributes:
    units:        px^2
    description:  Pupil Area

#### Session Metadata

In [66]:
session_mdata = json.loads((session_path / 'session.json').read_text())
session_mdata

{'date': '2016-12-14T00:00:00', 'subject': 'cori', 'researcher': 'steinmetz'}

### Merge Data

In [67]:
dset = xr.Dataset({
    'brain_region': brain_region_da,
    'lfp': lfp_da,
    'pupil_area': pupil_area,
    'pupil_x': pupil_x,
    'pupil_y': pupil_y,
    'wheel': wheel_da,
})
dset.attrs = session_mdata
dset

<xarray.Dataset> Size: 7MB
Dimensions:       (channel: 7, trial: 364, time: 250)
Coordinates:
  * channel       (channel) |S7 49B b'CHAN001' b'CHAN002' ... b'CHAN007'
  * trial         (trial) int64 3kB 0 1 2 3 4 5 6 ... 358 359 360 361 362 363
  * time          (time) uint32 1kB 0 10 20 30 40 ... 2450 2460 2470 2480 2490
Data variables:
    brain_region  (channel) object 56B 'ACA' 'LS' 'MOs' 'CA3' 'DG' 'SUB' 'VISp'
    lfp           (trial, time, channel) float64 5MB -2.851 0.9415 ... 5.571
    pupil_area    (trial, time) float64 728kB 0.1658 0.1587 ... 0.1099 0.1199
    pupil_x       (trial, time) float64 728kB 0.8129 0.7782 ... -0.486 -0.4916
    pupil_y       (trial, time) float64 728kB 0.6642 0.6128 ... 0.2392 0.27
    wheel         (trial, time) int8 91kB -1 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0
Attributes:
    date:        2016-12-14T00:00:00
    subject:     cori
    researcher:  steinmetz

In [72]:
merged_path = Path(f'data/merged')
merged_path.mkdir(exist_ok=True, parents=True)
dset.to_netcdf(merged_path / f"{session_path.name}.nc")